In [ ]:
import requests
import json
import time
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import zscore

API_KEY = '1a9ea75b5f92dfda381d817fc00e5458' 
USER_AGENT = '808e0a8bbaf9d30bf181edb60c742824'  
LIMIT = 20  ##Número de resultados a obtener

headers = {
    'user-agent': USER_AGENT
}

##sacar top artists
def get_top_artists():
    params = {
        'method': 'chart.getTopArtists',
        'api_key': API_KEY,
        'format': 'json',
        'limit': LIMIT
    }
    response = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['artists']['artist'])
    else:
        print(f"Error en la solicitud de top artistas globales: {response.status_code}")
    time.sleep(1)
    return df

##sacar top artistas por pais
def get_top_artists_by_country(country):
    params = {
        'method': 'geo.getTopArtists',
        'country': country,
        'api_key': API_KEY,
        'format': 'json',
        'limit': LIMIT
    }
    response = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['topartists']['artist'])
    else:
        print(f"Error en la solicitud de top artistas por país: {response.status_code}")
    time.sleep(1)
    return df

##sacar top canciones por pais
def get_top_tracks_by_country(country):
    params = {
        'method': 'geo.getTopTracks',
        'country': country,
        'api_key': API_KEY,
        'format': 'json',
        'limit': LIMIT
    }
    response = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['tracks']['track'])
    else:
        print(f"Error en la solicitud de top canciones por país: {response.status_code}")
    time.sleep(1)
    return df


##sacar top canciones de x artista
def get_top_tracks_by_artist(artist):
    
    params = {
        'method': 'artist.getTopTracks',
        'artist': artist,
        'api_key': API_KEY,
        'format': 'json',
        'limit': LIMIT
    }
    response = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['toptracks']['track'])
    else:
        print(f"Error en la solicitud de top álbumes del artista: {response.status_code}")
    time.sleep(1)
    return df
    
##limpia y ordena dataframe
def limpiar_y_ordenar(df, columnas_a_eliminar=None):
    if columnas_a_eliminar:
        df = df.drop(columns=columnas_a_eliminar)
    if 'listeners' in df.columns:
        df = df.sort_values(by='listeners', ascending=False)
    df = df.reset_index(drop=True)
    return df

In [ ]:
df_top_artist = get_top_artists()
df_top_artist_filtered = limpiar_y_ordenar(df_top_artist, ["mbid", "url", "streamable", "image"])
df_top_artist_filtered[["playcount", "listeners"]] = df_top_artist_filtered[["playcount", "listeners"]].astype(float)
df_top_artist_filtered.info()

In [ ]:
df_top_artist_filtered.describe().loc["mean", :]
##Estas son las medias de oyentes y reproducciones en el top global

In [ ]:
media_playcount = df_top_artist_filtered["playcount"].mean()
media_playcount

In [ ]:
media_listeners = df_top_artist_filtered["listeners"].mean()
media_listeners

In [ ]:
mediana_playcount = df_top_artist_filtered["playcount"].median()
mediana_playcount

In [ ]:
mediana_listeners = df_top_artist_filtered["listeners"].median()
mediana_listeners

In [ ]:
sns.barplot(x = "name", y = "playcount", data = df_top_artist_filtered)
plt.xticks(rotation=90)
plt.axhline(mediana_playcount, color='red', linestyle='--', label=f'Mediana: {mediana_playcount}')
plt.axhline(media_playcount, color='green', linestyle='--', label=f'Media: {media_playcount}')
plt.legend()
plt.show()

#La media y la mediana se encuentran en valores similares a la mayoria de artistas,
#pero podemos observar que los valores mas alejados se dan en Taylor Swift por arriba
#y Chappell Roan por abajo

In [ ]:
sns.barplot(x = "name", y = "listeners", data = df_top_artist_filtered)
plt.xticks(rotation=90)
plt.axhline(mediana_listeners, color='red', linestyle='--', label=f'Mediana: {mediana_listeners}')
plt.axhline(media_listeners, color='green', linestyle='--', label=f'Media: {media_listeners}')
plt.legend()
plt.title("Oyentes por Artista con Media y Mediana")
plt.xlabel("Artistas")
plt.ylabel("Oyentes")
plt.show()

#Los datos en cuanto a oyentes estan mas normalizados ya que no hay ningun artista que se
#aleje demasiado de la media y la mediana las cuales tienen un valor practicamente igual.
#En este caso la diferencia de oyentes por arriba y por abajo de la media es similar.

In [ ]:
listeners= df_top_artist_filtered["listeners"]
playcount= df_top_artist_filtered["playcount"]
stats.pearsonr(listeners, playcount)[0]

#La correlacion entre oyentes y reproducciones es muy baja en cuanto a estos artistas.

In [ ]:
sns.set(style='whitegrid')
sns.lineplot(data=df_top_artist_filtered, x='listeners', y='playcount', marker='o')

plt.title("Relación entre Reproducciones y Oyentes")
plt.ylabel('Oyentes')
plt.xlabel('Reproducciones')
plt.tight_layout()
plt.show()
#El número de oyentes de los artistas no influye directamente en el número de reproducciones.

In [ ]:
#TOP 20 CANCIONES POR PAIS
paises = ["Spain","France","Germany","United Kingdom","United States","Russian Federation"]
df_total = pd.DataFrame()

for pais in paises:
    df_tracks = get_top_tracks_by_country(pais)
    df_tracks_filtered = limpiar_y_ordenar(df_tracks,["mbid", "url", "image", "streamable.#text", "streamable.fulltrack", "artist.mbid", "artist.url", "@attr.rank"])
    df_tracks_filtered[["duration", "listeners"]] = df_tracks_filtered[["duration", "listeners"]].astype(float)
    df_tracks_filtered['pais'] = pais

    df_total = pd.concat(objs = [df_total, df_tracks_filtered], ignore_index = True).sort_values("listeners", ascending = False)
    
df_total

In [ ]:
estadisticas_listeners = df_total.groupby('pais')['listeners'].agg(['mean', 'median'])

estadisticas_listeners = estadisticas_listeners.reset_index()

df_melted = pd.melt(estadisticas_listeners, id_vars='pais', value_vars=['mean', 'median'],
                    var_name='Estadística', value_name='Listeners')

orden_paises = estadisticas_listeners.sort_values('mean', ascending=False)['pais']


sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
sns.barplot(data=df_melted, x='pais', y='Listeners', hue='Estadística', order=orden_paises)

plt.title('Media y Mediana de Listeners por País')
plt.xlabel('País')
plt.ylabel('Número de Listeners')
plt.xticks(rotation=45)
plt.legend(title='Estadística')

plt.tight_layout()
plt.show()

#La media y la mediana de oyentes en todos los países analizados es similar, solamente en United Kingdom
#se aprecia un descenso mas evidente que en la mediana que en la media.

In [ ]:
canciones_por_artista = df_total['artist.name'].value_counts()


plt.figure(figsize = (10, 6))
sns.barplot(x = canciones_por_artista.index, y = canciones_por_artista.values)
plt.xticks(rotation=90, fontsize=8)
plt.xlabel("Artistas")
plt.ylabel("Canciones en el TOP")
plt.title("Canciones por artista")
plt.show()

#En este gráfico podemos observar la cantidad de canciones en el top 20 de los paises analizados.
#Cabe destacar la presencia de Nirvana como el artista con mas canciones (12) estando presente
#en todos los países.

In [ ]:
pais = df_total['pais'].unique()
 
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 10))
axes = axes.flatten()
 
for i, pais in enumerate(pais):
    datos_pais = df_total[df_total['pais'] == pais]
    sns.barplot(x='listeners', y='name', data=datos_pais, ax=axes[i])
    axes[i].set_title(f'Oyentes de {pais}')
    axes[i].set_xlabel('Oyentes')
    axes[i].set_ylabel('Canción')
 
plt.tight_layout()
plt.show()

#Gráficas de oyentes por país. En todos los paises analizados "Smell Like Teen Spirit"
#se encuentra en el top 1 de oyentes.

In [ ]:
df_duration_media= df_total["duration"].map(lambda x : x if x > 0 else df_total["duration"].mean())

sns.kdeplot(x = df_duration_media,
            y = df_total["listeners"])

plt.xlabel("Duración")
plt.ylabel("Oyentes")
plt.title("Relación entre Oyentes y duración de canción")
plt.xticks(rotation=90)
plt.show()

#La relacion entre la duración y oyentes se concentra en los 200 y 300 segundos.
#A mayor duración de las canciones el numero de oyentes baja.

In [ ]:
sns.histplot(df_total["listeners"], color = "green", kde = True)

plt.xlabel("Oyentes")
plt.ylabel("Frecuencia")
plt.title("Frecuencia de Oyentes")
plt.show()

stats.skew(df_total["listeners"])
#Los oyentes tienen una asimetría negativa (cola a la izquierda)

In [ ]:
##Top canciones por artista
artistas = ["Lana del Rey", "Ic3peak", "Billie Eilish", "Sia", "Estopa", "Bad Bunny"]
df_total_artistas = pd.DataFrame()

for artista in artistas:
    df_tracks = get_top_tracks_by_artist(artista)
    
    columnas_deseadas = ["url", "streamable", "image", "artist.url", "@attr.rank", "mbid", "artist.mbid"]
    columnas_existentes = [col for col in columnas_deseadas if col in df_tracks.columns]
    
    df_tracks_filtered = limpiar_y_ordenar(df_tracks, columnas_existentes)
    
    for col in ["playcount", "listeners"]:
        if col in df_tracks_filtered.columns:
            df_tracks_filtered[col] = pd.to_numeric(df_tracks_filtered[col], errors='coerce')
    
    df_tracks_filtered.drop(columns=["mbid", "artist.mbid"], errors='ignore', inplace=True)
    
    df_total_artistas = pd.concat([df_total_artistas, df_tracks_filtered], ignore_index=True)

if "listeners" in df_total_artistas.columns:
    df_total_artistas = df_total_artistas.sort_values("listeners", ascending=False)

df_total_artistas

In [ ]:
reproducciones_por_artista = df_total_artistas.groupby('artist.name')['playcount'].sum().sort_values(ascending=False)

oyentes_por_artista = df_total_artistas.groupby('artist.name')['listeners'].sum().sort_values(ascending=False)

In [ ]:
##Gráfico de barras para reproducciones
sns.barplot(x=reproducciones_por_artista.values, y=reproducciones_por_artista.index)
plt.title('Reproducciones por Artista')
plt.xlabel('Reproducciones')
plt.ylabel('Artista')
plt.show()

##Gráfico de barras para oyentes
sns.barplot(x=oyentes_por_artista.values, y=oyentes_por_artista.index)
plt.title('Oyentes por Artista')
plt.xlabel('Oyentes')
plt.ylabel('Artista')
plt.show()

#Se observa el número de reproducciones y oyentes por artista en la que se destacan 
#notablemente "Lana del Rey" y "Billie Eilish".

In [ ]:
top_canciones = df_total_artistas.sort_values(by='playcount', ascending=False).head(10)

sns.barplot(x='playcount', y='name', data=top_canciones, hue='artist.name')
plt.title('Top 10 Canciones por Reproducciones')
plt.xlabel('Reproducciones')
plt.ylabel('Canción')
plt.legend(title='Artista')
plt.show()

#De los 6 artistas analizados extraemos el top 10 canciones y se ve como solo aparecen
#los dos artistas con mas oyentes y reproducciones.

In [ ]:
##correlacion de pearson
listeners= df_total_artistas["listeners"]
playcount= df_total_artistas["playcount"]
stats.pearsonr(listeners, playcount)[0]

#La correlacion entre oyentes y reproducciones es muy alta con estos 6 artistas a diferencia del top global.

In [ ]:
##relacion oyentes y reproducciones
sns.scatterplot(data=df_total_artistas, x='listeners', y='playcount', hue='artist.name')
plt.title('Oyentes vs Reproducciones por Canción')
plt.xlabel('Oyentes')
plt.ylabel('Reproducciones')
plt.legend(title='Artista')
plt.show()

#Esta gráfica apoya la correlacion entre oyentes y reproducciones. Se detecta un outlier
#en una cancion en concreto.

In [ ]:
mediana = df_total_artistas['Reproducciones_por_Oyente'].median()

##calcular IQR
Q1 = df_total_artistas['Reproducciones_por_Oyente'].quantile(0.25)
Q3 = df_total_artistas['Reproducciones_por_Oyente'].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites para los outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

##columna outlier
df_total_artistas['Es_Outlier'] = df_total_artistas['Reproducciones_por_Oyente'].apply(
    lambda x: 'Outlier' if x < limite_inferior or x > limite_superior else 'Normal')

sns.histplot(data=df_total_artistas, x='Reproducciones_por_Oyente', hue='Es_Outlier', bins=30, palette={'Normal': 'skyblue', 'Outlier': 'orange'}, multiple='stack')

##mediana
plt.axvline(mediana, color='green', linestyle='--', label=f'Mediana: {mediana:.2f}')
plt.axvline(x = Q1 - 1.5 * IQR, color = "coral", linestyle = "--", label = "Límite Q3")
plt.axvline(x = Q3 + 1.5 * IQR, color = "coral", linestyle = "--", label = "Límite Q1")


# Personalizar el gráfico
plt.title('Distribución de Reproducciones por Oyente con Outliers')
plt.xlabel('Reproducciones por Oyente')
plt.ylabel('Frecuencia')
plt.legend()
plt.show()

#Tras calcular el IQR hemos sacado los limites del rango intercuartil.
#Se puede detectar un outlier hacia la derecha por encima del limite superior.

In [ ]:
artistas = df_total_artistas['artist.name'].unique()

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 10))
axes = axes.flatten()

for i, artista in enumerate(artistas):
    datos_artista = df_total_artistas[df_total_artistas['artist.name'] == artista]
    sns.barplot(x='playcount', y='name', data=datos_artista, ax=axes[i])
    axes[i].set_title(f'Reproducciones de {artista}')
    axes[i].set_xlabel('Reproducciones')
    axes[i].set_ylabel('Canción')

plt.tight_layout()
plt.show()

#Reproducciones de canciones por artista.

In [ ]:
##Distribución por reproducciones
sns.histplot(df_total_artistas['playcount'], bins=20, kde=True)
plt.title('Distribución de Reproducciones')
plt.xlabel('Reproducciones')
plt.ylabel('Frecuencia')
plt.show()

##Distribución por oyentes
sns.histplot(df_total_artistas['listeners'], bins=20, kde=True)
plt.title('Distribución de Oyentes')
plt.xlabel('Oyentes')
plt.ylabel('Frecuencia')
plt.show()

#Distribuciones por oyentes y reproducciones. Ambas son bastantes irregulares
#y se ve un pico muy pronunciado.

In [ ]:
df_total_artistas['Reproducciones_Log'] = np.log1p(df_total_artistas['playcount'])
df_total_artistas['Oyentes_Log'] = np.log1p(df_total_artistas['listeners'])

##Distribución por reproducciones Log
sns.histplot(df_total_artistas['Reproducciones_Log'], bins=20, kde=True)
plt.title('Distribución de Reproducciones')
plt.xlabel('Reproducciones')
plt.ylabel('Frecuencia')
plt.show()

##Distribución por oyentes Log
sns.histplot(df_total_artistas['Oyentes_Log'], bins=20, kde=True)
plt.title('Distribución de Oyentes')
plt.xlabel('Oyentes')
plt.ylabel('Frecuencia')
plt.show()

#Tras realizar la distribución logarítmica las gráficas se han estabilizado algo más.

In [ ]:
##Top artistas por paises
paises = ["Spain","France","Germany","United Kingdom","United States","Russian Federation"]
df_total_pais = pd.DataFrame()

for pais in paises:
    df_tracks = get_top_artists_by_country(pais)
    df_tracks_filtered = limpiar_y_ordenar(df_tracks,["mbid", "url", "streamable", "image"])
    df_tracks_filtered["Country"] = pais
    df_tracks_filtered[["listeners"]] = df_tracks_filtered[["listeners"]].astype(float)

    df_total_pais = pd.concat(objs = [df_total_pais, df_tracks_filtered], ignore_index = True).sort_values("listeners", ascending = False)
    
df_total_pais

In [ ]:
estadisticas_listeners_2 = df_total_pais.groupby('Country')['listeners'].agg(['mean', 'median'])

estadisticas_listeners_2 = estadisticas_listeners_2.reset_index()

df_melted_2 = pd.melt(estadisticas_listeners_2, id_vars='Country', value_vars=['mean', 'median'],
                    var_name='Estadística', value_name='Listeners')

orden_paises_2 = estadisticas_listeners_2.sort_values('mean', ascending=False)['Country']


sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
sns.barplot(data=df_melted_2, x='Country', y='Listeners', hue='Estadística', order=orden_paises_2)

plt.title('Media y Mediana de Listeners por País')
plt.xlabel('País')
plt.ylabel('Número de Listeners')
plt.xticks(rotation=45)
plt.legend(title='Estadística')

plt.tight_layout()
plt.show()

#La media y la mediana de oyentes en todos los países analizados es similar, solamente en United Kingdom
#se aprecia un descenso mas evidente que en la mediana que en la media.

In [ ]:
df_canciones_por_pais = df_total_pais.groupby('Country')['listeners'].count().reset_index()

df_canciones_por_pais = df_canciones_por_pais.sort_values('listeners', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x = "Country", y = "listeners", hue = "Country", data = df_total_pais, palette = "Paired", legend = False)

plt.xlabel('Número de canciones')
plt.ylabel('País')
plt.title('Número de canciones por país')

plt.show()

#Oyentes por país. Los datos de Russia son menos fiables ya que la línea es más grande.

In [ ]:
df_total_pais["Oyentes_Log"] = np.log1p(df_total_pais["listeners"])

plt.figure(figsize = (10, 6))
sns.boxplot(x = "Country", y = "Oyentes_Log", hue = "Country", legend = False, data = df_total_pais, palette = "Paired")

plt.xlabel("Paises")
plt.ylabel("Log (Oyentes)")
plt.title("Oyentes por País Log")
plt.grid()

plt.show()

#Despues de aplicar el logaritmo a la distribución, observamos que en Russia la 
#mayoría de datos se encuentran por debajo de la mediana y también se ve
#un outlier por el limite inferior. En los demás países se aprecia una distribución
#normal.

In [ ]:
media_oyentes = df_total_pais["listeners"].mean()
mediana_oyentes = df_total_pais["listeners"].median()

plt.figure(figsize =(10, 10))
plt.barh(df_total_pais["name"], df_total_pais["listeners"], color="steelblue")

plt.axvline(x = media_oyentes, color = "yellowgreen", linestyle = "--", label = f"Media: {int(media_oyentes)}")
plt.axvline(x = mediana_oyentes, color = "coral", linestyle = "--", label = f"Mediana: {int(mediana_oyentes)}")

plt.xlabel("Oyentes")
plt.ylabel("Artista")
plt.title("Oyentes por Artista con Media y Mediana")
plt.legend()
plt.grid()

plt.show()

#La mayoría de artista estan por debajo de la media y hay cinco artistas que la 
#superan bastante haciendo que esta sea mayor.

In [ ]:
df_total_pais["Z_Score"] = zscore(df_total_pais["listeners"])

plt.figure(figsize = (10, 10))
sns.barplot(y = df_total_pais["name"], 
            x = df_total_pais["Z_Score"], 
            hue = df_total_pais["name"], 
            legend = False, palette = "Spectral")

plt.axvline(x = 2.5, color = "coral", linestyle = "--", label = "Outlier (>3)")
plt.axvline(x = -2.5, color = "coral", linestyle = "--", label = "Outlier (<-3)")
plt.axvline(x = 0, color = "yellowgreen", linestyle = "--", label = "Media (Z-score = 0)")

plt.xlabel("Oyentes")
plt.ylabel("Artistas")
plt.title("Oyentes por Artista con Z-score")
plt.grid(axis = "x")
plt.legend()

plt.show()

#Tras realizar el z-score y marcar los limites se observa un outlier por la zona 
#del límite inferior el cual es сплин.

In [ ]:
countries = df_total_pais["Country"].unique()

fig, axes = plt.subplots(nrows = 2, ncols = 3, figsize = (18, 10))
axes = axes.flatten()

for i, country in enumerate(countries):
    df_country = df_total_pais[df_total_pais["Country"] == country]
    
    sns.barplot(x = "listeners", y = "name", hue = "Country", data = df_country, ax = axes[i], palette = "viridis", legend = False)
    axes[i].set_title(f"Oyentes por artista en {country}")
    axes[i].set_xlabel("Oyentes")
    axes[i].set_ylabel("Artistas")

plt.tight_layout()
plt.show()

#Oyentespor artista en cada país.

In [ ]:
df_total_listeners_by_artist = df_total_pais.groupby("name", as_index=False)["listeners"].sum()

df_total_listeners_by_artist = df_total_listeners_by_artist.sort_values("listeners", ascending=False)

plt.figure(figsize=(10, 10))
sns.barplot(x="listeners", y="name", hue = "name", data = df_total_listeners_by_artist, palette = "Spectral", legend = False)

plt.xlabel("Oyentes Totales")
plt.ylabel("Artistas")
plt.title("Oyentes Totales por Artista")
plt.grid(axis="x")

plt.show()

#Oyentes totales por artista. Coldplay y Radiohead lideran esta gráfica.